In [1]:
import os, glob
path_files_texts = []
path_to_dir_of_path_reports = "/Users/tp53/Documents/code/rgci_kundan_chufal/sample_hpe"

# Loop through all the text files in the directory and store contents in a giant list
for file in glob.glob(os.path.join(path_to_dir_of_path_reports, "*.txt")):
    # Open each file and read its contents into a string
    with open(file, "r" , encoding='cp1252') as f:
        text_contents_of_file = f.read()
        # Append the string to the list
        path_files_texts.append(text_contents_of_file)

In [4]:
import os
print(os.getcwd())

/Users/tp53/Documents/code/rgci_kundan_chufal


In [7]:
import cohere
import numpy as np

apiKey = 'acYUhCGgSu54zqgZDFrrJtgLsNFYMF59L0992b6I'
co = cohere.Client(apiKey)


embeds = co.embed(texts=path_files_texts,                  				
  					model="small",
  					truncate="START").embeddings

np.save('cohere_embeddings_small.npy', embeds)


In [17]:
import umap.umap_ as umap
import pandas as pd
import altair as alt


'''
import cohere
import numpy as np
import re

from tqdm import tqdm
from datasets import load_dataset


from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans
from bertopic._ctfidf import ClassTFIDF
from sklearn.feature_extraction.text import CountVectorizer

'''

reducer = umap.UMAP(n_neighbors=50) 
umap_embeds = reducer.fit_transform(embeds)

df = pd.DataFrame(path_files_texts)
print(df.shape)

df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

print(df.shape)

'''
# Plot
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    )#,
    #tooltip=['title']
).configure(background="#FDF7F0"
).properties(
    width=700,
    height=400,
    title='Pathology reports'
)

chart.interactive()
'''

(74, 1)


'\n# Plot\nchart = alt.Chart(df).mark_circle(size=60).encode(\n    x=#\'x\',\n    alt.X(\'x\',\n        scale=alt.Scale(zero=False),\n        axis=alt.Axis(labels=False, ticks=False, domain=False)\n    ),\n    y=\n    alt.Y(\'y\',\n        scale=alt.Scale(zero=False),\n        axis=alt.Axis(labels=False, ticks=False, domain=False)\n    )#,\n    #tooltip=[\'title\']\n).configure(background="#FDF7F0"\n).properties(\n    width=700,\n    height=400,\n    title=\'Pathology reports\'\n)\n\nchart.interactive()\n'